In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from geopy.distance import great_circle as vc 
import math as Math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import keras 
# import tensorflow
# import tensorflow as tf
from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# from keras.layers import LSTM
import math, time
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
# from keras.utils.vis_utils import plot_model
# from keras.layers import Bidirectional
# from keras.utils.generic_utils import get_custom_objects
import random
import os
import sys
sys.path.insert(0,"/content/sample_data/")
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm.notebook import tqdm




class load_data(Dataset):
    def __init__(self, x, y):
        self.length = x.shape[0]
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.length


# A simple utility function for converting pytorch tensors to numpy
def tensor_to_numpy(data, device_type="cpu"):
    if device_type == "cpu":
        return data.detach().numpy()
    else:
        return data.cpu().detach().numpy()


# A simple utility function for converting numpy to pytorch tensors
def numpy_to_tensor(data, dtype="float", device="cuda"):
    if dtype=="float":
        return torch.tensor(data, dtype=torch.float).to(device)
    elif dtype=="long":
        return torch.tensor(data, dtype=torch.long).to(device)





seed = 1024
def reset_random_seeds():
   print("resetting seeds")
   os.environ['PYTHONHASHSEED']=str(seed)
#    tensorflow.random.set_seed(seed)
   np.random.seed(seed)
   random.seed(seed)
   torch.manual_seed(seed)

reset_random_seeds()

#Supressing Warnings
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

path = '/content/sample_data/' # Path of file
filePath = path + 'IMDDataTillLandfall_moreFeatures2.csv'
data = pd.read_csv(filePath, index_col=0)

# data.dropna()

faniData = data[data['cyclone_id'].str.contains('2019-2')]
print('faniData Shape', faniData.shape)
# bulbulData = data[data['cyclone_id'].str.contains('2019-9')]
# print('bulbulData Shape', bulbulData.shape)
# gajaData = data[data['cyclone_id'].str.contains('2018-13')]
# print('gajaData Shape', gajaData.shape)

#Removing Fani, Bulbul, Gaja data from dataset
data = data[~data['cyclone_id'].str.contains('2019_2')]
# data = data[~data['cyclone_id'].str.contains('2019_9')]
# data = data[~data['cyclone_id'].str.contains('2018_13')]

print('data  Shape', data.shape)

# Normalize the values to predict them more easily in our model.
scalers = {}
# columnsToNormalize = ['ecp','sst','distance','direction','lati','long']
# columnsToNormalize = ['ecp','sst','lati','grade', 'pd', 'cino']

# 24 steps GC
# columnsToNormalize = ['season', 'lati', 'pd', 'grade', 'sst', 'distance', 'direction']
# columnsToNormalize = ['season', 'lati', 'cino', 'pd', 'grade', 'sst', 'distance', 'direction']

# 2 steps GC
# columnsToNormalize = ['lati', 'cino', 'pd', 'grade', 'sst', 'distance', 'direction']

# mutual information
columnsToNormalize = ['lati', 'long', 'cino', 'ecp', 'pd', 'grade', 'sst']



# columnsToNormalize = ['long','cino', 'ecp', 'pd', 'grade', 'sst', 'distance' ]


for col in columnsToNormalize:
  scaler = StandardScaler()
  data[[col]] = scaler.fit_transform(data[[col]])
  scalers[col] =  scaler

data_pad = []

for key in np.unique(data['cyclone_id']):
    data_pad += [data[ data.loc[:, 'cyclone_id'] == key].loc[:, ['lati', 'long', 'cino', 'ecp', 'pd', 'grade', 'sst', 'mssw']].values]

for col in columnsToNormalize:
  scaler = scalers[col]
  faniData[[col]] = scaler.transform(faniData[[col]])

data_padF = []
data_padF += [faniData.loc[:, ['lati', 'long', 'cino', 'ecp', 'pd', 'grade', 'sst', 'mssw']].values]

# for col in columnsToNormalize:
#   scaler = scalers[col]
#   bulbulData[[col]] = scaler.transform(bulbulData[[col]])

# data_padB = []
# data_padB += [bulbulData.loc[:, ['ecp','sst','distance','direction','lati','long','mssw']].values]

# for col in columnsToNormalize:
#   scaler = scalers[col]
#   gajaData[[col]] = scaler.transform(gajaData[[col]])

# data_padG = []
# data_padG += [gajaData.loc[:, ['ecp','sst','distance','direction','lati','long','mssw']].values]

def load_dataTime(data_pad, trainPoints ,amount_of_features):
    X = []
    y = []
    for cdata in data_pad:
      length = len(cdata)
      diff = length - trainPoints
      if diff > 0:
        hours = diff *3.0 
        for i in range(diff):
          cc = cdata[i : i + trainPoints]
          yy = hours - i*3.0
          X.append(cc)
          y.append(yy)
    X = np.array(X)
    y = np.array(y)
    x_train = np.reshape(X, (X.shape[0], X.shape[1], amount_of_features)) 
    #y_train = np.reshape(y, (y.shape[0], y.shape[1], 1))
    return [x_train, y]

def load_dataInt(data_pad, trainPoints ,amount_of_features):
    X = []
    y = []
    for cdata in data_pad:
      length = len(cdata)
      diff = length - trainPoints
      if diff > 0:
        yy = cdata[length-1][-1] 
        for i in range(diff):
          cc = cdata[i : i + trainPoints]
          X.append(cc)
          y.append(yy)
    X = np.array(X)
    y = np.array(y)
    x_train = np.reshape(X, (X.shape[0], X.shape[1], amount_of_features)) 
    #y_train = np.reshape(y, (y.shape[0], y.shape[1], 1))
    return [x_train, y]

def build_model(feature_count, loader_train, device, epochs):
    model = HSM1(device, feature_count, 64)
    model = model.to(device)
    print(model.parameters)
    criterion = nn.MSELoss()
    # criterion = nn.HuberLoss()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


    for epoch in tqdm(range(epochs)):
        # print("\n\nEpoch: ", epoch+1, "\n-----------------")
        running_loss = 0
        
        for index, data in enumerate(loader_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            #print(inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs).squeeze()
            # Compute loss for each timestep
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()
            running_loss+= loss.item()
        if (epoch+1)%10 == 0:
            print("Training loss: ", running_loss/len(loader_train))
    return model


def get_pred(model, device, loader):
    actual, preds = list(), list()
    # Make sure no gradients are being used during evaluation
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            labels = tensor_to_numpy(labels, device)
            pred = model(inputs).squeeze()
            pred = tensor_to_numpy(pred, device)
            actual.extend(labels)
            preds.extend(pred)
    return actual, preds

# swish activation
def new(x):
    return 0.5*x*(1 + torch.tanh(x))
    #return x/(1+tf.exp(-2*x))

# import torch.nn.functional as F


# # mish activation
# def new(x):
#     return 1.5*x* (torch.tanh(F.softplus(x)))
#     # return x* (torch.tanh(F.softplus(x)))
#     #return x/(1+tf.exp(-2*x))

     
# get_custom_objects().update({'new': New(new)})

feature_count = 8
trains = [4, 6, 8, 12]
# trains = [12]

resetting seeds
faniData Shape (58, 14)
data  Shape (3485, 14)


# New Section

# Dilated RNN

In [2]:
class HSM1_layer(nn.Module):
    def __init__(self, dilation, input_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=None, return_sequences=True):
        # Define all the variables required
        super(HSM1_layer, self).__init__()
        self.dilation = dilation
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.use_gru = use_gru
        self.use_lstm = use_lstm
        self.use_rnn = use_rnn
        self.device = device
        self.return_sequences = return_sequences

        if self.use_gru == True:
            self.rnn = nn.GRU(self.input_dim, self.hidden_dim, batch_first=True)
        elif self.use_lstm == True:
            self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, batch_first=True)
        elif self.use_rnn == True:
            self.rnn = nn.RNN(self.input_dim, self.hidden_dim, batch_first=True)

    def forward(self, data):
        final_out = self._cell_output(data, self.rnn)
        if not self.return_sequences:
            return final_out[:,-1,:]
        else:
            return final_out

    def _cell_output(self, data, cell):
        output = self.drnn_layer(cell, data)
        return output
    
    def drnn_layer(self, cell, data):
        batch_size = data.shape[0]
        n_steps = data.shape[1]
        hidden_size = cell.hidden_size

        #print("\n======================= DILATION:", self.dilation, "=====================================")

        data = self._pad_inputs(data, batch_size, n_steps)
        #print("After Padding:", data.shape)
        data = self._prepare_inputs(data)
        data = self._apply_cell(data, cell)
        #print("After Applying cell:", data.shape)
        data = self._split_outputs(data, batch_size, hidden_size)
        #print("After Splitting outputs:", data.shape)
        outputs = self._unpad_outputs(data, n_steps)
        #print("After Unpadding:", outputs.shape)
        #print("\n")

        return outputs
    
    def _apply_cell(self, data, cell):
        data, _ = cell(data)
        return data

    def _unpad_outputs(self, data, n_steps):
        return data[:,:n_steps,:]

    def _split_outputs(self, data, batch_size, hidden_size):
        n_steps = data.shape[1]*self.dilation
        numerator = data.shape[1]
        new_data = torch.reshape(data, (batch_size, n_steps, hidden_size))
        new_data = torch.cat([new_data[:, i::numerator, :] for i in range(numerator)], dim=1)
        return new_data

    def _pad_inputs(self, data, batch_size, n_steps):
        is_even = (n_steps % self.dilation) == 0

        if not is_even:
            dilated_steps = n_steps // self.dilation + 1
            zeros_ = torch.zeros(batch_size, dilated_steps * self.dilation - data.shape[1], data.shape[2]).to(self.device)
            data = torch.cat((data, zeros_), dim=1)
        else:
            dilated_steps = n_steps // self.dilation

        return data

    def _prepare_inputs(self, data):
        batch_size = data.shape[0]
        new_data = torch.cat([data[:, i::self.dilation, :] for i in range(self.dilation)], dim=1)
        
        return torch.reshape(new_data, (self.dilation*batch_size, data.shape[1]//self.dilation, data.shape[2]))

# Initialize Model

In [3]:
# Create an HSM1 model
class HSM1(nn.Module):
    def __init__(self, device, input_dim, hidden_dim):
        super(HSM1, self).__init__()
        self.layer1 = HSM1_layer(1, input_dim, 2048,  use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=True)
        self.layer2 = HSM1_layer(2, 2048, 1024,  use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=True)
        self.layer3 = HSM1_layer(4, 1024, 512, use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=False)
        # self.layer4 = HSM1_layer(8, 512, 256, use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=False)
        # self.layer5 = HSM1_layer(16, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer6 = HSM1_layer(32, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer7 = HSM1_layer(64, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer8 = HSM1_layer(128, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer9 = HSM1_layer(256, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=False)
        self.linear = nn.Linear(512, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = new(x)
        x = self.layer2(x)
        x = new(x)
        x = self.layer3(x)
        x = new(x)
        # x = self.layer4(x)
        # x = new(x)
        # x = self.layer5(x)
        # x = self.layer6(x)
        # x = self.layer7(x)
        # x = self.layer8(x)
        # x = self.layer9(x)
        x = self.linear(x)
        return x

# Remaining Code

In [4]:
def getMaeRmse(predictions, y_test):

  testScore = mean_squared_error(predictions, y_test)
  testScoreMae = mean_absolute_error(predictions, y_test)

  return np.sqrt(testScore), testScoreMae

In [ ]:
for i in range(len(trains)):
  trainPoints = trains[i]
  print("===========================================================")
  print("trainPoints (T) = ", trainPoints)
  X, y = load_dataTime(data_pad, trainPoints, feature_count)
  XF, yF = load_dataTime(data_padF, trainPoints, feature_count)
  XG, yG = load_dataTime(data_padG, trainPoints, feature_count)
  XB, yB = load_dataTime(data_padB, trainPoints, feature_count)

  datasetF = load_data(numpy_to_tensor(XF, "float", device), numpy_to_tensor(yF, "float", device))
  datasetG = load_data(numpy_to_tensor(XG, "float", device), numpy_to_tensor(yG, "float", device))
  datasetB = load_data(numpy_to_tensor(XB, "float", device), numpy_to_tensor(yB, "float", device))

  loaderF = DataLoader(dataset=datasetF, batch_size=256, shuffle=False, num_workers=0)
  loaderG = DataLoader(dataset=datasetG, batch_size=512, shuffle=False, num_workers=0)
  loaderB = DataLoader(dataset=datasetB, batch_size=512, shuffle=False, num_workers=0)

  X, y = shuffle(X, y, random_state=seed)
  print("No of train points", X.shape)

  kf = KFold(n_splits=5)

  scores = []
  scoresF = []
  scoresG = []
  scoresB = []
  
  scoresMae = []
  scoresFMae = []
  scoresGMae = []
  scoresBMae = []

  for train_index, test_index in kf.split(X):
      X_train = X[train_index]
      y_train = y[train_index]
      X_test = X[test_index]
      y_test = y[test_index]

      dataset_train = load_data(numpy_to_tensor(X_train, "float", device), numpy_to_tensor(y_train, "float", device))
      dataset_test = load_data(numpy_to_tensor(X_test, "float", device), numpy_to_tensor(y_test, "float", device))

      loader_train = DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=0)
      loader_test = DataLoader(dataset=dataset_test, batch_size=512, shuffle=False, num_workers=0)

      model = build_model(feature_count, loader_train, device, epochs=400)

      y_test, predictions = get_pred(model, device, loader_test)
      yF, predictionsF = get_pred(model, device, loaderF)
      yG, predictionsG = get_pred(model, device, loaderG)
      yB, predictionsB = get_pred(model, device, loaderB)

      rmse, mae = getMaeRmse(predictions, y_test)
      rmseF, maeF = getMaeRmse(predictionsF, yF)
      rmseG, maeG = getMaeRmse(predictionsG, yG)
      rmseB, maeB  = getMaeRmse(predictionsB, yB)

      scores.append(rmse)
      scoresF.append(rmseF)
      scoresG.append(rmseG)
      scoresB.append(rmseB)

      scoresMae.append(mae)
      scoresFMae.append(maeF)
      scoresGMae.append(maeG)
      scoresBMae.append(maeB)
      print("scores RMSE ", scores)
      print("scores Mae   ", scoresMae)

  print("5 fold Time to Landfall RMSE ",np.array(scores).mean())
  print("5 fold Fani Time to Landfall RMSE  ", np.array(scoresF).mean())
  print("5 fold Gaja Time to Landfall RMSE  ", np.array(scoresG).mean())
  print("5 fold Bulbul Time to Landfall RMSE  ", np.array(scoresB).mean())
    
  print("5 fold Time to Landfall MAE  ",np.array(scoresMae).mean())
  print("5 fold Fani Time to Landfall MAE  ", np.array(scoresFMae).mean())
  print("5 fold Gaja Time to Landfall MAE   ", np.array(scoresGMae).mean())
  print("5 fold Bulbul Time to Landfall MAE  ", np.array(scoresBMae).mean())
    

trainPoints (T) =  4
No of train points (3189, 4, 7)
<bound method Module.parameters of HSM1(
  (layer1): HSM1_layer(
    (rnn): GRU(7, 2048, batch_first=True)
  )
  (layer2): HSM1_layer(
    (rnn): GRU(2048, 1024, batch_first=True)
  )
  (layer3): HSM1_layer(
    (rnn): GRU(1024, 512, batch_first=True)
  )
  (linear): Linear(in_features=512, out_features=1, bias=True)
)>


  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  2493.144241333008
Training loss:  2017.264111328125
Training loss:  1754.1929809570313
Training loss:  1616.8552825927734
Training loss:  1557.924040222168
Training loss:  1536.982925415039
Training loss:  1533.5357513427734
Training loss:  1534.0845153808593
Training loss:  1530.4940704345704
Training loss:  1529.1703002929687
Training loss:  1224.0983505249023
Training loss:  1140.551710510254
Training loss:  905.6720642089844
Training loss:  606.885099029541
Training loss:  467.75628490447997
Training loss:  342.135009765625
Training loss:  257.9918737411499
Training loss:  197.10047469139099
Training loss:  152.34449882507323
Training loss:  123.12011342048645
Training loss:  102.98347537517547
Training loss:  82.41023926734924
Training loss:  69.47494480609893
Training loss:  56.255358684062955
Training loss:  48.017793250083926
Training loss:  40.31041406393051
Training loss:  33.5914024412632
Training loss:  65.94280242919922
Training loss:  23.949883988499643
Tr

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  2628.1727142333984
Training loss:  2131.9487182617186
Training loss:  1845.8397750854492
Training loss:  1696.41515045166
Training loss:  1627.8604431152344
Training loss:  1606.0981719970703
Training loss:  1595.060025024414
Training loss:  1595.9571105957032
Training loss:  1592.7450149536132
Training loss:  1593.2051666259765
Training loss:  1381.3522964477538
Training loss:  1177.3085327148438
Training loss:  975.7025711059571
Training loss:  690.8348499298096
Training loss:  502.5257987976074
Training loss:  382.80419425964357
Training loss:  281.75077896118165
Training loss:  212.13382663726807
Training loss:  163.13408455848693
Training loss:  130.99729866981505
Training loss:  106.3558933019638
Training loss:  91.81764965057373
Training loss:  74.20551573038101
Training loss:  60.33912702202797
Training loss:  50.00123490691185
Training loss:  48.998766887187955
Training loss:  34.729972833395
Training loss:  30.611801809072496
Training loss:  28.150389021635057

...

Training loss:  2402.1237041766826
Training loss:  2034.1774573692908
Training loss:  1797.930424616887
Training loss:  1609.4925114558293
Training loss:  1463.9530757023738
Training loss:  1383.3176410381611
Training loss:  1347.1117694561299
Training loss:  1310.5526498647837
Training loss:  1313.4519254244292
Training loss:  1292.677762545072
Training loss:  1302.2613619290864
Training loss:  1312.3060044508713
Training loss:  1306.2096205491287
Training loss:  1309.8119178185095
Training loss:  1157.6495713454026
Training loss:  1026.0196075439453
Training loss:  932.4441598745493
Training loss:  807.6161146897537
Training loss:  597.6957104022687
Training loss:  494.43274982158954
Training loss:  383.4155000539926
Training loss:  294.18188799344574
Training loss:  242.46599886967584
Training loss:  206.88585178668683
Training loss:  173.77637518369235
Training loss:  147.72242742318375
Training loss:  136.39024463066687
Training loss:  118.7740687590379
Training loss:  96.90848540

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  2261.9854407677285
Training loss:  1948.9794029822717
Training loss:  1697.6760934682993
Training loss:  1502.8938903808594
Training loss:  1419.4221778282752
Training loss:  1356.4916569636418
Training loss:  1306.2561856783354
Training loss:  1285.2301471416768
Training loss:  1274.0999168982873
Training loss:  1260.90965623122
Training loss:  1262.688485952524
Training loss:  1282.552018385667
Training loss:  1254.365720308744
Training loss:  1179.5612159142127
Training loss:  1099.1370286207932
Training loss:  1032.698469895583
Training loss:  915.1636634239784
Training loss:  771.908467219426
Training loss:  696.2540459266075
Training loss:  576.59107032189
Training loss:  479.5155962430514
Training loss:  424.3531987116887
Training loss:  338.99775549081653
Training loss:  274.3106962350699
Training loss:  262.91929391714245
Training loss:  193.28798983647272
Training loss:  129.7821066196148
Training loss:  110.19245391625624
Training loss:  94.49308982262245
Tra

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  2313.54450401893
Training loss:  1963.6708655724158
Training loss:  1729.2801372821514
Training loss:  1550.8652813251201
Training loss:  1436.6782930814302
Training loss:  1346.9232201209436
Training loss:  1322.399169921875
Training loss:  1297.5073030911958
Training loss:  1280.7240318885217
Training loss:  1284.2784588153545
Training loss:  1262.521000788762
Training loss:  1262.3451819786658
Training loss:  1254.5355694110576
Training loss:  1250.306884765625
Training loss:  1121.417008620042
Training loss:  1059.156231219952
Training loss:  1006.2978703425481
Training loss:  904.9678321251503
Training loss:  727.0281618558444
Training loss:  562.5495957594651
Training loss:  454.19627439058746
Training loss:  359.9111615694486
Training loss:  265.63302157475397


In [5]:
for i in range(len(trains)):
  trainPoints = trains[i]
  print("===========================================================")
  print("trainPoints (T) = ", trainPoints)
  X, y = load_dataInt(data_pad, trainPoints, feature_count)
  XF, yF = load_dataInt(data_padF, trainPoints, feature_count)
  # XG, yG = load_dataInt(data_padG, trainPoints, feature_count)
  # XB, yB = load_dataInt(data_padB, trainPoints, feature_count)

  datasetF = load_data(numpy_to_tensor(XF, "float", device), numpy_to_tensor(yF, "float", device))
  
  loaderF = DataLoader(dataset=datasetF, batch_size=256, shuffle=False, num_workers=0)



  X, y = shuffle(X, y, random_state=0)
  print("No of train points", X.shape)

  kf = KFold(n_splits=5)

  scores = []
  scoresF = []
  # scoresG = []
  # scoresB = []
  
  scoresMae = []
  scoresFMae = []
  # scoresGMae = []
  # scoresBMae = []

  for train_index, test_index in kf.split(X):
      X_train = X[train_index]
      y_train = y[train_index]
      X_test = X[test_index]
      y_test = y[test_index]

      dataset_train = load_data(numpy_to_tensor(X_train, "float", device), numpy_to_tensor(y_train, "float", device))
      dataset_test = load_data(numpy_to_tensor(X_test, "float", device), numpy_to_tensor(y_test, "float", device))

      loader_train = DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=0)
      loader_test = DataLoader(dataset=dataset_test, batch_size=512, shuffle=False, num_workers=0)
      
      model = build_model(feature_count, loader_train, device, epochs=400)

      y_test, predictions = get_pred(model, device, loader_test)

      yF, predictionsF = get_pred(model, device, loaderF)

      rmse, mae = getMaeRmse(predictions, y_test)
      rmseF, maeF = getMaeRmse(predictionsF, yF)

      scores.append(rmse)
      scoresF.append(rmseF)

      scoresMae.append(mae)
      scoresFMae.append(maeF)

      print("scores RMSE ", scores)
      print("scores Mae   ", scoresMae)

  print("5 fold Intensity at Landfall RMSE ",np.array(scores).mean())
  print("5 fold Fani Intensity at Landfall RMSE  ", np.array(scoresF).mean())
  # print("5 fold Gaja Intensity at Landfall RMSE  ", np.array(scoresG).mean())
  # print("5 fold Bulbul Intensity at Landfall RMSE  ", np.array(scoresB).mean())
    
  print("5 fold Intensity at Landfall MAE  ",np.array(scoresMae).mean())
  print("5 fold Fani Intensity at Landfall MAE  ", np.array(scoresFMae).mean())
  # print("5 fold Gaja Intensity at Landfall MAE   ", np.array(scoresGMae).mean())
  # print("5 fold Bulbul Intensity at Landfall MAE  ", np.array(scoresBMae).mean())
    

trainPoints (T) =  4
No of train points (2820, 4, 8)
<bound method Module.parameters of HSM1(
  (layer1): HSM1_layer(
    (rnn): GRU(8, 2048, batch_first=True)
  )
  (layer2): HSM1_layer(
    (rnn): GRU(2048, 1024, batch_first=True)
  )
  (layer3): HSM1_layer(
    (rnn): GRU(1024, 512, batch_first=True)
  )
  (linear): Linear(in_features=512, out_features=1, bias=True)
)>


  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  653.4732309977213
Training loss:  626.7125371297201
Training loss:  2145.8822224934897
Training loss:  1406.0138397216797
Training loss:  956.3273298475477
Training loss:  758.8467017279731
Training loss:  683.1485705905491
Training loss:  673.5067291259766
Training loss:  651.5341169569227
Training loss:  635.9871181911892
Training loss:  635.44005923801
Training loss:  480.8809831407335
Training loss:  341.1088769700792
Training loss:  257.4992387559679
Training loss:  179.50468338860406
Training loss:  112.37244701385498
Training loss:  78.79044490390353
Training loss:  53.51703401406606
Training loss:  37.42719978756375
Training loss:  25.294575691223145
Training loss:  25.6784561475118
Training loss:  14.132645143402947
Training loss:  34.07168241341909
Training loss:  19.61394923263126
Training loss:  2.4162032388978534
Training loss:  3.8864707549413047
Training loss:  1.778594371345308
Training loss:  28.183596690495808
Training loss:  2.497564322418637
Training

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  651.0978096856011
Training loss:  612.2754092746311
Training loss:  412.8570302327474
Training loss:  326.6706136067708
Training loss:  248.93385908338757
Training loss:  222.24782201978894
Training loss:  136.60973962148032
Training loss:  91.52672571606107
Training loss:  60.38505999247233
Training loss:  126.13891961839464
Training loss:  25.01542920536465
Training loss:  27.68664512369368
Training loss:  16.934151940875584
Training loss:  17.688506934377884
Training loss:  8.646283328533173
Training loss:  11.46913558906979
Training loss:  19.170572174919975
Training loss:  4.082790652910869
Training loss:  1.9094745814800262
Training loss:  3.6571262809965344
Training loss:  81.40658336215549
Training loss:  26.947937991884018
Training loss:  2.5578745106856027
Training loss:  1.4108696348137326
Training loss:  2.2163134962320328
Training loss:  1.117503322660923
Training loss:  89.65906567043729
Training loss:  11.318658590316772
Training loss:  1.1602284113566081

...

Training loss:  706.5867360432943
Training loss:  666.0306282043457
Training loss:  713.6965611775717
Training loss:  588.1211293538412
Training loss:  492.9641908009847
Training loss:  438.34703890482587
Training loss:  415.8369903564453
Training loss:  396.2376346588135
Training loss:  326.82970174153644
Training loss:  328.99495188395184
Training loss:  346.6562150319417
Training loss:  325.16848305861157
Training loss:  188.4127581914266
Training loss:  142.31656261285147
Training loss:  107.63349135716756
Training loss:  67.25690793991089
Training loss:  60.46732691923777
Training loss:  53.239690194527306
Training loss:  62.005510091781616
Training loss:  37.47653520107269
Training loss:  241.3517567316691
Training loss:  46.325967152913414
Training loss:  14.243695026884476
Training loss:  8.415108399776122
Training loss:  4.751943363497655
Training loss:  3.749823182821274
Training loss:  2.080321077567836
Training loss:  1.802456379091988
Training loss:  1.4378229772361617
Tra

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  676.3795216878256
Training loss:  661.3526980082194
Training loss:  682.468194325765
Training loss:  568.1537497838339
Training loss:  437.99228922526044
Training loss:  383.97437540690106
Training loss:  315.36921374003094
Training loss:  260.1943168640137
Training loss:  246.54111544291177
Training loss:  150.04280853271484
Training loss:  105.78742623329163
Training loss:  79.88930130004883
Training loss:  51.07165511449178
Training loss:  29.650189836819965
Training loss:  20.83194561799367
Training loss:  23.5114422639211
Training loss:  120.32157723108928
Training loss:  55.11575905481974
Training loss:  15.918539683024088
Training loss:  16.075985074043274
Training loss:  4.312812998890877
Training loss:  2.715107781191667
Training loss:  3.068908919890722
Training loss:  228.80347442626953
Training loss:  21.906045575936634
Training loss:  10.22794622182846
Training loss:  1.8427510932087898
Training loss:  1.3986199523011844
Training loss:  1.465925091256698
Tr

  0%|          | 0/400 [00:00<?, ?it/s]

Training loss:  696.9477189381918
Training loss:  694.5387649536133
Training loss:  570.7576967875162
Training loss:  456.51574389139813
Training loss:  411.38227971394855
Training loss:  327.47316487630206
Training loss:  274.5940179824829
Training loss:  217.7327737212181
Training loss:  158.80150922139487
Training loss:  148.24256165822348
Training loss:  116.3049243291219
Training loss:  86.2939437230428
Training loss:  118.63224093119304
Training loss:  40.80681610107422
Training loss:  50.69066826502482
Training loss:  24.791503856579464
Training loss:  95.81726201375325
Training loss:  23.650535677870113
Training loss:  11.741642634073893
Training loss:  9.766932050387064
Training loss:  40.80828748146693
Training loss:  83.48434925079346
Training loss:  11.545100271701813
Training loss:  5.677302777767181
Training loss:  4.1092242648204165
Training loss:  4.312022145837545
Training loss:  4.778269564112027
Training loss:  1.686902140577634
Training loss:  14.54985123872757
Trai

In [ ]:
trainPoints = 8
X, y = load_dataTime(data_pad, trainPoints, feature_count)
XF, yF = load_dataTime(data_padF, trainPoints, feature_count)
XG, yG = load_dataTime(data_padG, trainPoints, feature_count)
XB, yB = load_dataTime(data_padB, trainPoints, feature_count)
X, y = shuffle(X, y, random_state=0)

X1, y1 = load_dataInt(data_pad, trainPoints, feature_count)
XF1, yF1 = load_dataInt(data_padF, trainPoints, feature_count)
XG1, yG1 = load_dataInt(data_padG, trainPoints, feature_count)
XB1, yB1 = load_dataInt(data_padB, trainPoints, feature_count)
X1, y1 = shuffle(X1, y1, random_state=0)

model = build_model(feature_count, trainPoints)
model1 = build_model(feature_count, trainPoints)

model.fit(X, y, batch_size=512, epochs=150, verbose=0, shuffle = True)
model1.fit(X1, y1, batch_size=512, epochs=150, verbose=0, shuffle = True)

predictionsF = model.predict(XF)
predictionsG = model.predict(XG)
predictionsB = model.predict(XB)

predictionsF1 = model1.predict(XF1)
predictionsG1 = model1.predict(XG1)
predictionsB1 = model1.predict(XB1)

In [ ]:
#Time and Intensity Plot for Fani
print("trainPoints  ", trainPoints)

lengthF = len(yF)

plt.figure(figsize=(10, 10))
fig, ax1 = plt.subplots(figsize=(12, 4))
a = np.linspace(1,lengthF,lengthF)*3 + 8*3

color = 'tab:red'
ax1.set_xlabel('Hour (during cyclone progress)')
ax1.set_ylabel('Time to Landfall (Hours)', color=color)
ax1.plot(a, yF, marker = '*',label = "Actual time to Landfall",color = 'red')
ax1.plot(a, predictionsF, marker = '+',label = "Predicted time to Landfall",color ='green')
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_ticks(a)
plt.xticks(rotation=60)
plt.ylim((0,160))
ax1.legend(loc=1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Intensity (MSSW) at Landgall (in knots)', color=color)  # we already handled the x-label with ax1
ax2.plot(a, yF1, marker = '*',label = "Actual intensity at Landfall",color = 'blue')
ax2.plot(a, predictionsF1, marker = '+',label = "intensity Longitude at Landfall",color='magenta')

ax2.tick_params(axis='y', labelcolor=color)
plt.ylim((82,88))
fig.tight_layout()  # otherwise the right y-label is slightly clipped
ax2.legend(loc=3)
plt.show()

In [ ]:
#Time and Intensity Plot for Gaja
print("trainPoints  ", trainPoints)

lengthG = len(yG)

plt.figure(figsize=(10, 10))
fig, ax1 = plt.subplots(figsize=(12, 4))
a = np.linspace(1,lengthG,lengthG)*3 + 8*3

color = 'tab:red'
ax1.set_xlabel('Hour (during cyclone progress)')
ax1.set_ylabel('Time to Landfall (Hours)', color=color)
ax1.plot(a, yG, marker = '*',label = "Actual time to Landfall",color = 'red')
ax1.plot(a, predictionsG, marker = '+',label = "Predicted time to Landfall",color ='green')
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_ticks(a)
plt.xticks(rotation=60)
plt.ylim((0,160))
ax1.legend(loc=1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Intensity (MSSW) at Landgall (in knots)', color=color)  # we already handled the x-label with ax1
ax2.plot(a, yG1, marker = '*',label = "Actual intensity at Landfall",color = 'blue')
ax2.plot(a, predictionsG1, marker = '+',label = "intensity Longitude at Landfall",color='magenta')

ax2.tick_params(axis='y', labelcolor=color)
plt.ylim((67,73))
fig.tight_layout()  # otherwise the right y-label is slightly clipped
ax2.legend(loc=3)
plt.show()

In [ ]:
#Time and Intensity Plot for Bulbul
print("trainPoints  ", trainPoints)

lengthB = len(yB)

plt.figure(figsize=(10, 10))
fig, ax1 = plt.subplots(figsize=(12, 4))
a = np.linspace(1,lengthB,lengthB)*3 + 8*3

color = 'tab:red'
ax1.set_xlabel('Hour (during cyclone progress)')
ax1.set_ylabel('Time to Landfall (Hours)', color=color)
ax1.plot(a, yB, marker = '*',label = "Actual time to Landfall",color = 'red')
ax1.plot(a, predictionsB, marker = '+',label = "Predicted time to Landfall",color ='green')
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_ticks(a)
plt.xticks(rotation=60)
plt.ylim((0,120))
ax1.legend(loc=1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Intensity (MSSW) at Landgall (in knots)', color=color)  # we already handled the x-label with ax1
ax2.plot(a, yB1, marker = '*',label = "Actual intensity at Landfall",color = 'blue')
ax2.plot(a, predictionsB1, marker = '+',label = "intensity Longitude at Landfall",color='magenta')

ax2.tick_params(axis='y', labelcolor=color)
plt.ylim((29,31.5))
fig.tight_layout()  # otherwise the right y-label is slightly clipped
ax2.legend(loc=3)
plt.show()

In [ ]:
#Time to calculate 150 epochs
X, y = load_dataTime(data_pad, 4, feature_count)
X, y = shuffle(X, y, random_state=0)
model = build_model(feature_count, 4)
start = time.time()
model.fit(X, y, batch_size=512, epochs=150, verbose=0, shuffle = True)
end = time.time()
print("time (seconds) ", end-start)

In [ ]:
model = build_model(feature_count, 4)
X, y = load_dataTime(data_pad, 4, feature_count)
model.fit(X, y, batch_size=512, epochs=10, verbose=0, shuffle = True)
plot_model(model, to_file='model2.png', show_shapes=True, show_layer_names=False)